In [24]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import pickle
import os, re
print(os.listdir("../input"))
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

['testData.tsv', 'sampleSubmission.csv', 'labeledTrainData.tsv', 'unlabeledTrainData.tsv']


In [2]:
train = pd.read_csv("../input/labeledTrainData.tsv", header = 0, delimiter = '\t')
test = pd.read_csv("../input/testData.tsv", header = 0, delimiter = '\t')

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

1. Define some pre-processing functions

In [4]:
def html_to_text(review):
    """Return extracted text string from provided HTML string."""
    review_text = BeautifulSoup(review, "lxml").get_text()
    if len(review_text) == 0:
        review_text = review
    review_text = re.sub(r"\<.*\>", "", review_text)
    try:
        review_text = review_text.encode('ascii', 'ignore').decode('ascii')#ignore \xc3 etc.
    except UnicodeDecodeError:
        review_text = review_text.decode("ascii", "ignore")
    return review_text


def letters_only(text):
    """Return input string with only letters (no punctuation, no numbers)."""
    # It is probably worth experimenting with milder prepreocessing (eg just removing punctuation)
    return re.sub("[^a-zA-Z]", " ", text)

def rnn_tokenizer_review_preprocess(review):
    """Preprocessing used before fitting/transforming RNN tokenizer - Html->text, remove punctuation/#s, lowercase."""
    return letters_only(html_to_text(review)).lower()

In [5]:
def get_train_val_data(reviews_to_features_fn=None, df = train):
    #df = pd.read_csv('labeledTrainData.tsv', header=0, quotechar='"', sep='\t')
    SEED = 1000
    # Shuffle data frame rows
    np.random.seed(SEED)
    df = df.iloc[np.random.permutation(len(df))]

    if reviews_to_features_fn:
        feature_rows = df["review"].map(reviews_to_features_fn)
        if type(feature_rows[0]) == np.ndarray:
            num_instances = len(feature_rows)
            num_features = len(feature_rows[0])
            x = np.concatenate(feature_rows.values).reshape((num_instances, num_features))
        else:
            x = feature_rows
    else:
        x = df["review"]

    y = df["sentiment"]

    # Split 80/20
    test_start_index = int(df.shape[0] * .8)
    x_train = x[0:test_start_index]
    y_train = y[0:test_start_index]
    x_val = x[test_start_index:]
    y_val = y[test_start_index:]

    return x_train, y_train, x_val, y_val

## Propcessed the data

In [6]:
x_train, y_train, x_val, y_val = get_train_val_data(rnn_tokenizer_review_preprocess)
x_test = test["review"].map(rnn_tokenizer_review_preprocess)
y_test = test["sentiment"]

## Generate the text sequence for RNN model

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [8]:
np.random.seed(1000)
num_most_freq_words_to_include = 5000
MAX_REVIEW_LENGTH_FOR_KERAS_RNN = 500
embedding_vector_length = 32

In [9]:
train_review_list = x_train.tolist()
val_review_list = x_val.tolist()
test_review_list = x_test.tolist()
all_review_list = x_train.tolist() + x_val.tolist()

In [10]:
tokenizer = Tokenizer(num_words=num_most_freq_words_to_include)
tokenizer.fit_on_texts(all_review_list)

In [11]:
train_reviews_tokenized = tokenizer.texts_to_sequences(train_review_list)
x_train = pad_sequences(train_reviews_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
val_review_tokenized = tokenizer.texts_to_sequences(val_review_list)
x_val = pad_sequences(val_review_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
test_review_tokenized = tokenizer.texts_to_sequences(test_review_list)
x_test = pad_sequences(test_review_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)

## Architecture the RNN Model



In [12]:
from keras.layers import Input, Embedding, Dropout, Conv1D, MaxPool1D, GRU, LSTM, Dense
from keras.models import Model

In [13]:
def rnn_model(use_cnn = True, use_lstm = False):
    input_sequences = Input(shape = (MAX_REVIEW_LENGTH_FOR_KERAS_RNN,))
    initial_dropout = 0.2
    embedding_layer = Embedding(input_dim = num_most_freq_words_to_include, 
                                output_dim = embedding_vector_length,
                                input_length = MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
    X = embedding_layer(input_sequences)
    X = Dropout(0.2)(X)
    if use_cnn:
        X = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(X)
        X = MaxPool1D(pool_size=2)(X)
        
    # Add GRU layers
    dropout_W = 0.0
    dropout_U = 0.0
    
    if use_lstm:
        X = LSTM(100, dropout = dropout_W, recurrent_dropout = dropout_U)(X)
    else:
        X = GRU(100, dropout=dropout_W, recurrent_dropout=dropout_U)(X)
    X = Dropout(0.2)(X)
    outputs= Dense(1, activation='sigmoid')(X)
    model = Model(inputs = input_sequences, outputs = outputs)
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

## GRU Model

In [ ]:
gru_model = rnn_model(use_lstm=False)

In [ ]:
gru_model.summary()

In [ ]:
gru_model.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=[x_val, y_val])

In [ ]:
y_test_pred_gru = gru_model.predict(x_test)

In [30]:
print("Accuracy on test set using lstm: %0.3f%%"%(accuracy_score(y_test, y_test_pred_gru.round())*100))
print("Precision on test set using lstm: %0.3f"%(precision_score(y_test, y_test_pred_gru.round())))
print("Recall on test setusing lstm: %0.3f"%(recall_score(y_test, y_test_pred_gru.round())))
print("F1-Score on test set using lstm: %0.3f"%(f1_score(y_test, y_test_pred_gru.round())))
print("F1-Score on test set using lstm: %0.3f"%(f1_score(y_test, y_test_pred_gru.round()))

SyntaxError: unexpected EOF while parsing (<ipython-input-30-1236e7bff10e>, line 5)

## LSTM Model

In [14]:
lstm_model = rnn_model(use_lstm=True)
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [15]:
lstm_model.fit(x_train, y_train, batch_size = 64, epochs = 3, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 181s 9ms/step - loss: 0.4813 - acc: 0.7428 - val_loss: 0.3607 - val_acc: 0.8398
Epoch 2/3
20000/20000 [==============================] - 176s 9ms/step - loss: 0.2717 - acc: 0.8926 - val_loss: 0.2807 - val_acc: 0.8852
Epoch 3/3
20000/20000 [==============================] - 175s 9ms/step - loss: 0.2208 - acc: 0.9173 - val_loss: 0.2991 - val_acc: 0.8908


In [31]:
y_test_pred_lstm = lstm_model.predict(x_test)

### Evaluate the Model Performance on Test Data

In [ ]:
print("Accuracy on test set using lstm: %0.3f%%"%(accuracy_score(y_test, y_test_pred_lstm.round())*100))
print("Precision on test set using lstm: %0.3f"%(precision_score(y_test, y_test_pred_lstm.round())))
print("Recall on test setusing lstm: %0.3f"%(recall_score(y_test, y_test_pred_lstm.round())))
print("F1-Score on test set using lstm: %0.3f"%(f1_score(y_test, y_test_pred_lstm.round())))
print("F1-Score on test set using lstm: %0.3f"%(f1_score(y_test, y_test_pred_lstm.round()))

Overall, GRU model performs slightly well than LSTM, but the difference is very small.